# Quantitative analyses
In this notebook, I include all of my quantitative anlayses/results.

## Setup
To pull from the GitHub repository in Colab:
```
%cd drive/MyDrive/Conservation\ Research/Code/counting-cranes
!git add .
!git stash
!git pull
```
----

In [ ]:
#Mounting Google Drive...
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Will have to restart runtime after running this cell!
!pip install -r "/content/drive/MyDrive/Conservation Research/Code/counting-cranes/requirements.txt"

     |████████████████████████████████| 804.1 MB 2.7 kB/s 
     |████████████████████████████████| 17.4 MB 128 kB/s 
     |████████████████████████████████| 7.1 MB 20.8 MB/s 
     |████████████████████████████████| 2.2 MB 38.7 MB/s 
     |████████████████████████████████| 17.4 MB 152 kB/s 
     |████████████████████████████████| 98 kB 6.8 MB/s 
     |████████████████████████████████| 15.3 MB 130 kB/s 
     |████████████████████████████████| 11.5 MB 9.1 MB/s 
     |████████████████████████████████| 806 kB 43.4 MB/s 
     |████████████████████████████████| 8.3 MB 14.0 MB/s 
     |████████████████████████████████| 27.4 MB 110 kB/s 
     |████████████████████████████████| 51.0 MB 56 kB/s 
     |████████████████████████████████| 165 kB 49.0 MB/s 
     |████████████████████████████████| 47.6 MB 41 kB/s 
     |████████████████████████████████| 132 kB 52.4 MB/s 
     |████████████████████████████████| 636 kB 44.5 MB/s 
     |████████████████████████████████| 329 kB 54.9 MB/s 
     |███████████

In [ ]:
import json

config = json.load(open('/content/drive/MyDrive/Conservation Research/Code/counting-cranes/config.json', 'r'))
DATA_FP = config['data_filepath_colab']
CODE_FP = config['code_filepath_colab']
MODEL_SAVE_FP = config['model_saves_filepath_colab']
SEEDS = config['random_seeds']
FIXED_PERM = config['fixed_data_permutation']
ASPDNET_HYPERPARAMETERS = config['ASPDNet_params']
FASTER_RCNN_HYPERPARAMETERS = config['faster_rcnn_params']
tile_size = tuple(config['tile_size'])

In [ ]:
import sys
import os
sys.path.append(CODE_FP) 
sys.path.append(os.path.join(CODE_FP, 'density_estimation'))
sys.path.append(os.path.join(CODE_FP, 'object_detection'))
sys.path.append(os.path.join(CODE_FP, 'density_estimation', 'ASPDNet'))

from bird_dataset import *
from utils import *
from ASPDNet_model import *
from ASPDNet.model import ASPDNet
from faster_rcnn_model import *

import torch
from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything
import pandas as pd
from itertools import chain

## General stuff

In [ ]:
#Choosing the data split
MODE = input('Which data split should we use? ')

if MODE == 'fixed_perm':
  indices = FIXED_PERM
  save_name_aspdnet = 'ASPDNet_custom_permutation_8.17.2021.ckpt'
  save_name_frcnn = 'faster_rcnn_custom_permutation_7.30.2021.pth'
elif MODE == 'seed_1': 
  seed_everything(SEEDS[0])
  indices = torch.randperm(40).tolist()
  save_name_aspdnet = 'ASPDNet_seed_1693_8.18.2021.ckpt'
  save_name_frcnn = 'faster_rcnn_seed_1693_8.16.2021.pth'
elif MODE == 'seed_2': 
  seed_everything(SEEDS[1])
  indices = torch.randperm(40).tolist()
  save_name_aspdnet = 'ASPDNet_seed_2021_8.19.2021.ckpt'
  save_name_frcnn = 'faster_rcnn_seed_2021_8.17.2021.pth'

Which data split should we use? seed_2


Global seed set to 2021


In [ ]:
#Establishing the datasets/dataloaders
bird_dataset_eval_aspdnet = BirdDataset(root_dir = DATA_FP, 
                                        transforms = get_transforms('density_estimation', train = False), 
                                        tiling_method = 'w_o_overlap', 
                                        annotation_mode = 'points', 
                                        tile_size = tile_size,
                                        sigma = 3)
bird_dataset_eval_frcnn = BirdDataset(root_dir = DATA_FP, 
                                      transforms = get_transforms('object_detection', train = False), 
                                      tiling_method = 'w_o_overlap', 
                                      annotation_mode = 'bboxes',
                                      tile_size = tile_size) 

#  recovering the test set
print(f'Indices used: {indices[28 : ]}')
dataset_test_aspdnet = torch.utils.data.Subset(bird_dataset_eval_aspdnet, indices[28 : ])
dataset_test_frcnn = torch.utils.data.Subset(bird_dataset_eval_frcnn, indices[28 : ]) 

#  wrapping datasets in dataloaders
dataloader_test_aspdnet = DataLoader(dataset_test_aspdnet, batch_size = ASPDNET_HYPERPARAMETERS['batch_size'], shuffle = False, collate_fn = collate_tiles_density) 
dataloader_test_frcnn = DataLoader(dataset_test_frcnn, batch_size = FASTER_RCNN_HYPERPARAMETERS['batch_size'], shuffle = False, collate_fn = collate_tiles_object_detection) 

Indices used: [19, 35, 9, 37, 22, 28, 15, 11, 2, 20, 33, 17]


In [ ]:
#Loading both models
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#  ASPDNet...
aspdnet = ASPDNet(allow_neg_densities = False).to(device)
pl_model_aspdnet = ASPDNetLightning.load_from_checkpoint(os.path.join(MODEL_SAVE_FP, 'ASPDNet', 'final_models', save_name_aspdnet), model = aspdnet)
pl_model_aspdnet.model.eval()

#  Faster R-CNN...
frcnn = get_faster_rcnn(backbone = 'ResNet50', num_classes = 2, **FASTER_RCNN_HYPERPARAMETERS['constructor_hyperparams']).to(device) 
frcnn.load_state_dict(torch.load(os.path.join(MODEL_SAVE_FP, 'faster_rcnn', 'final_models', save_name_frcnn)))
pl_model_frcnn = FasterRCNNLightning(model = frcnn)
pl_model_frcnn.model.eval();

## **Performance at different AGLs**
How does performance change at different flight heights?

In [ ]:
#Prepping the metadata stuff... to keep track of pred/true counts and AGL
image_metadata = pd.read_csv(os.path.join(DATA_FP, 'dataset_metadata.csv'), index_col = 0) #pulling in the metadata file

#  grabbing the names of the test images
test_indices = indices[28 : ]
test_image_names = list(np.array(bird_dataset_eval_frcnn.image_fps)[test_indices])

#  getting metadata for just the test images
test_metadata = image_metadata[image_metadata['Image_ID'].str.upper().isin([fp.upper() for fp in test_image_names])]

In [ ]:
test_metadata

,Image_ID,AGL_feet,AGL_m,GSD_cm,num_bboxes
0,FLIR2_20210321_204704_024_4313.tif,2500.0,761.962816,10.583333,2044
4,FLIR2_20210321_212535_841_5638.tif,2000.0,609.570253,8.466667,1426
6,FLIR2_20210321_213439_159_5907.tif,3500.0,1066.747943,14.816667,1400
8,FLIR2_20210321_215413_167_6212.tif,3000.0,914.355379,12.700000,2082
9,FLIR2_20210321_223143_100_6724.tif,2000.0,609.570253,8.466667,1821
11,FLIR2_20210321_223549_810_6783.tif,2000.0,609.570253,8.466667,1754
17,FLIR2_20210321_224952_670_7004.tif,3500.0,1066.747943,14.816667,116
22,FLIR2_20210321_225956_449_7159.tif,2500.0,761.962816,10.583333,1343
24,FLIR2_20210321_225959_452_7162.tif,2500.0,761.962816,10.583333,1023
26,FLIR2_20210321_230002_445_7165.tif,2500.0,761.962816,10.583333,360


### _Faster R-CNN_ 

In [ ]:
#Predicting on all imagery and recording true/pred count
img_performances = {}

with torch.no_grad():
  for tiles, targets, image_names, _ in dataloader_test_frcnn:
    tiles = [t.to(device) for t in tiles]

    preds = pl_model_frcnn(tiles)
    true_bboxes = [t['boxes'] for t in targets]
    pred_bboxes = [pred_dict['boxes'] for pred_dict in preds]
    
    pred_count = len(list(chain(*pred_bboxes)))
    true_count = len(list(chain(*true_bboxes)))

    img_name = '_'.join(image_names[0].split('_')[ : -1]) + '.TIF'
    img_performances[img_name] = (pred_count, true_count)

### _ASPDNet_

In [ ]:
#Predicting on all imagery and recording true/pred count
img_performances = {}

with torch.no_grad():
  for content_1, content_2 in zip(dataloader_test_aspdnet, dataloader_test_frcnn):
    tiles, densities, counts = content_1
    _, _, image_names, _ = content_2
    tiles = tiles.to(device)

    preds = pl_model_aspdnet(tiles)   
    pred_count = float(preds.sum())
    true_count = sum(counts)

    img_name = '_'.join(image_names[0].split('_')[ : -1]) + '.TIF'
    img_performances[img_name] = (pred_count, true_count)

In [ ]:
#Adding these values to the CSV
test_metadata = test_metadata.sort_values('Image_ID', axis = 0)

performance_tuples = [img_performances[k] for k in sorted(img_performances.keys())]
test_metadata['pred_counts'] = [p[0] for p in performance_tuples]
test_metadata['true_counts'] = [p[1] for p in performance_tuples]

test_metadata = test_metadata.drop('num_bboxes', axis = 1)
test_metadata

,Image_ID,AGL_feet,AGL_m,GSD_cm,pred_counts,true_counts
36,20180320_213531_728_9420.tif,2500.0,761.962816,21.166667,1673.760254,2298
35,20180321_223216_097_2818.tif,2000.0,609.570253,16.933333,1552.987549,1758
0,FLIR2_20210321_204704_024_4313.tif,2500.0,761.962816,10.583333,2563.170654,2042
4,FLIR2_20210321_212535_841_5638.tif,2000.0,609.570253,8.466667,1461.142578,1424
6,FLIR2_20210321_213439_159_5907.tif,3500.0,1066.747943,14.816667,1288.474121,1399
8,FLIR2_20210321_215413_167_6212.tif,3000.0,914.355379,12.700000,2066.955811,2082
9,FLIR2_20210321_223143_100_6724.tif,2000.0,609.570253,8.466667,1814.920410,1820
11,FLIR2_20210321_223549_810_6783.tif,2000.0,609.570253,8.466667,1326.572266,1752
17,FLIR2_20210321_224952_670_7004.tif,3500.0,1066.747943,14.816667,158.209320,116
22,FLIR2_20210321_225956_449_7159.tif,2500.0,761.962816,10.583333,1394.825928,1343


In [ ]:
#Saving the results
test_metadata.to_csv('aspdnet_seed=2021_error_results.csv')

In [ ]:
#CHECK: we should be able to recover MPE/RMSE/MAE metrics here...
pc = test_metadata['pred_counts']
tc = test_metadata['true_counts']

pe = 100 * np.absolute(pc - tc) / tc
ae = np.absolute(pc - tc)
se = (pc - tc) ** 2

print(f'MPE: {pe.mean()}\nMAE: {ae.mean()}\nRMSE: {se.mean() ** 0.5}')

MPE: 14.767591111720094
MAE: 187.95342254638672
RMSE: 277.99348922046744


## **Performance in 2018 vs 2021 imagery**
How does performance on the 2021 imagery match up against 2018 imagery performance?

*(Here, I'm just adding to the existing metadata/predictions dataframe!)*

In [ ]:
def sort_2018_v_2021(image_fp):
  return 2021 if image_fp.startswith('FLIR2') else 2018

test_metadata['year_taken'] = test_metadata['Image_ID'].apply(sort_2018_v_2021)
test_metadata

,Image_ID,AGL_feet,AGL_m,GSD_cm,pred_counts,true_counts,year_taken
38,20180320_212958_600_9152.tif,2500.0,761.962816,21.166667,2272.503418,1928,2018
36,20180320_213531_728_9420.tif,2500.0,761.962816,21.166667,1893.421387,2298,2018
30,20180321_220440_395_1832.tif,2000.0,609.570253,16.933333,2426.168701,2882,2018
35,20180321_223216_097_2818.tif,2000.0,609.570253,16.933333,1919.305420,1758,2018
29,FLIR2_20210321_201851_358_2510.TIF,3500.0,1066.747943,14.816667,4273.573242,4283,2021
2,FLIR2_20210321_205850_355_4974.tif,2500.0,761.962816,10.583333,1613.900024,1742,2021
10,FLIR2_20210321_223145_103_6726.tif,2000.0,609.570253,8.466667,1925.172852,1862,2021
14,FLIR2_20210321_223624_804_6818.tif,2000.0,609.570253,8.466667,2138.026611,2158,2021
17,FLIR2_20210321_224952_670_7004.tif,3500.0,1066.747943,14.816667,122.833420,116,2021
19,FLIR2_20210321_225423_648_7070.tif,3000.0,914.355379,12.700000,136.565857,190,2021


In [ ]:
#Saving the CSV...
test_metadata.to_csv('quantitative_results_ASPDNet.csv')